# Processing SAGE Phoenix data with Aurora

This is an example of how to process data collected with a Phoenix MTU-5c data with Aurora.  For now this assumes that an MTH5 has already been created. 

## Process With Aurora

In [1]:
# Required imports for the program.
%matplotlib widget

from pathlib import Path
import warnings

from aurora.config.config_creator import ConfigCreator
from aurora.pipelines.process_mth5 import process_mth5
from aurora.pipelines.run_summary import RunSummary
from aurora.transfer_function.kernel_dataset import KernelDataset

from mth5.helpers import close_open_files
from mtpy import MT

warnings.filterwarnings("ignore")

## Set paths

Set path the the `local` and `remote` reference stations, both paths and IDs.

In [2]:
local_station = "102"
local_mth5_path = Path(r"c:\Users\jpeacock\OneDrive - DOI\MTData\SAGE2023\102_060923\mth5_from_phoenix.h5")

remote_station = None
remote_mth5_path = None


## Create Run Summary

Create a run summary for a single sample rate for local and remote stations.


In [3]:
close_open_files()

sample_rate = 150
mth5_run_summary = RunSummary()
if remote_mth5_path is not None:
    mth5_run_summary.from_mth5s([local_mth5_path, remote_mth5_path])
else:
    mth5_run_summary.from_mth5s([local_mth5_path])
run_summary = mth5_run_summary.clone()
run_summary.add_duration()
run_summary.df = run_summary.df[run_summary.df.sample_rate == sample_rate].iloc[:]
run_summary.mini_summary

2023-06-30T11:54:00 [line 758] mth5.mth5.MTH5.close_mth5 - INFO: Flushing and closing c:\Users\jpeacock\OneDrive - DOI\MTData\SAGE2023\102_060923\mth5_from_phoenix.h5


,survey,station_id,run_id,start,end
0,VAL,102,sr150_0001,2023-06-07 10:45:32+00:00,2023-06-09 09:45:39.993333333+00:00


## Create Kernel Dataset

This sets the runs for Aurora to process.  

In [4]:
kernel_dataset = KernelDataset()
if remote_station is not None:
    kernel_dataset.from_run_summary(run_summary, zen_station, rr_zen_station)
else:
    kernel_dataset.from_run_summary(run_summary, local_station)
mimimum_run_duration = 1  # seconds
kernel_dataset.drop_runs_shorter_than(mimimum_run_duration)
kernel_dataset.mini_summary

,survey,station_id,run_id,start,end,duration
0,VAL,102,sr150_0001,2023-06-07 10:45:32+00:00,2023-06-09 09:45:39.993333333+00:00,169207.993333


## Create Aurora Configuration

Create the Aurora configuration file.  These are the parameters to tell Aurora how to run.

In [5]:
cc = ConfigCreator()
config = cc.create_from_kernel_dataset(
    kernel_dataset,
    emtf_band_file=r"c:\Users\jpeacock\OneDrive - DOI\Documents\GitHub\aurora\aurora\config\emtf_band_setup\bs_six_level.cfg",
)

# need to update the channel nomenclature to what phoenix uses
config.channel_nomenclature.ex = "e1"
config.channel_nomenclature.ey = "e2"
config.channel_nomenclature.hx = "h1"
config.channel_nomenclature.hy = "h2"
config.channel_nomenclature.hz = "h3"

for decimation in config.decimations:
    if remote_station is not None:
        decimation.estimator.engine = "RME_RR"
    else:
        decimation.estimator.engine = "RME"
    decimation.window.type = "dpss"
    decimation.window.additional_args = {"alpha":2.5}
    decimation.output_channels = ["e1", "e2", "h3"]
    decimation.input_channels = ["h1", "h2"]
    decimation.window.overlap = 64
    decimation.window.num_samples = 128

c:\Users\jpeacock\OneDrive - DOI\Documents\GitHub\aurora\aurora\config\emtf_band_setup\bs_six_level.cfg


## Process

Now that everything is set up, estimate a transfer functions and create an EDI and figure.

In [6]:
%%time
tf_cls = process_mth5(
    config,
    kernel_dataset,
    units="MT",
    show_plot=False,
    z_file_path=None,
)
print("=== FINISHED ===")

edi = tf_cls.write(local_mth5_path.parent.joinpath(f"{local_station}_{sample_rate}.edi"))

# plot with MTpy
mt_obj = MT()
mt_obj.read(edi.fn)
p1 = mt_obj.plot_mt_response(fig_num=1, plot_num=2)
p1.save_plot(local_mth5_path.parent.joinpath(f"{edi.fn.stem}.png"), fig_dpi=300, close_plot=False)

2023-06-30 11:54:25.961 | INFO     | aurora.pipelines.transfer_function_kernel:memory_warning:227 - Total memory: 15.88 GB
2023-06-30 11:54:25.963 | INFO     | aurora.pipelines.transfer_function_kernel:memory_warning:231 - Total Bytes of Raw Data: 0.189 GB
2023-06-30 11:54:25.965 | INFO     | aurora.pipelines.transfer_function_kernel:memory_warning:234 - Raw Data will use: 1.191 % of memory
2023-06-30 11:54:40.061 | INFO     | aurora.transfer_function.kernel_dataset:initialize_dataframe_for_processing:338 - DATASET POPULATED
2023-06-30 11:54:40.109 | INFO     | aurora.pipelines.process_mth5:process_mth5:314 - Processing config indicates 6 decimation levels 
2023-06-30 11:54:40.113 | DEBUG    | aurora.pipelines.transfer_function_kernel:valid_decimations:173 - After validation there are 6 valid decimation levels
2023-06-30 11:54:40.114 | DEBUG    | aurora.pipelines.process_mth5:update_dataset_df:252 - DATASET DF UPDATED
2023-06-30 11:55:38.325 | WARNING  | aurora.pipelines.time_series_he

2023-06-30T11:55:40 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 0.114803s
2023-06-30T11:56:50 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 0.090326s
2023-06-30T11:57:57 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 0.071673s
2023-06-30T11:59:13 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 0.055518s
2023-06-30T12:01:20 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 0.042079s
2023-06-30T12:04:23 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 0.029786s


2023-06-30 12:08:34.775 | INFO     | aurora.pipelines.process_mth5:update_dataset_df:239 - DECIMATION LEVEL 1
2023-06-30 12:08:39.423 | DEBUG    | aurora.pipelines.process_mth5:update_dataset_df:252 - DATASET DF UPDATED
2023-06-30 12:08:50.725 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:08:50.783 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:08:50.845 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:08:50.900 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:08:50.953 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS


2023-06-30T12:08:51 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 0.631126s
2023-06-30T12:09:10 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 0.459211s
2023-06-30T12:09:27 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 0.286693s
2023-06-30T12:10:00 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 0.222071s
2023-06-30T12:10:35 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 0.168316s


2023-06-30 12:11:14.532 | INFO     | aurora.pipelines.process_mth5:update_dataset_df:239 - DECIMATION LEVEL 2
2023-06-30 12:11:15.452 | DEBUG    | aurora.pipelines.process_mth5:update_dataset_df:252 - DATASET DF UPDATED
2023-06-30 12:11:18.563 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:11:18.610 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:11:18.642 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:11:18.679 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:11:18.710 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS


...

2023-06-30 12:12:00.927 | INFO     | aurora.pipelines.process_mth5:update_dataset_df:239 - DECIMATION LEVEL 4
2023-06-30 12:12:01.021 | DEBUG    | aurora.pipelines.process_mth5:update_dataset_df:252 - DATASET DF UPDATED
2023-06-30 12:12:01.501 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:12:01.533 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:12:01.560 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:12:01.593 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:12:01.619 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS


2023-06-30T12:12:01 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 40.392036s
2023-06-30T12:12:02 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 27.799705s
2023-06-30T12:12:03 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 19.289901s
2023-06-30T12:12:04 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 14.212575s


2023-06-30 12:12:05.876 | INFO     | aurora.pipelines.process_mth5:update_dataset_df:239 - DECIMATION LEVEL 5
2023-06-30 12:12:05.922 | DEBUG    | aurora.pipelines.process_mth5:update_dataset_df:252 - DATASET DF UPDATED
2023-06-30 12:12:06.307 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:12:06.331 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:12:06.358 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:12:06.385 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS
2023-06-30 12:12:06.410 | WARNING  | aurora.pipelines.time_series_helpers:calibrate_stft_obj:275 - UNEXPECTED CHANNEL WITH NO FILTERS


2023-06-30T12:12:06 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 161.568142s
2023-06-30T12:12:06 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 111.198822s
2023-06-30T12:12:06 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 77.159602s
2023-06-30T12:12:06 [line 133] aurora.transfer_function_helpers.get_band_for_tf_estimate - INFO: Processing band 56.850300s


AttributeError: 'RunGroup' object has no attribute 'station_group'